In [1]:
import pandas as pd
import numpy as np

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
import numpy as np

# 模型定义
class AnimeRatingPredictor(nn.Module):
    def __init__(self, input_size, hidden_sizes=[512, 256, 128], dropout=0.3):
        """
        全连接神经网络，适配输入维度
        :param input_size: 输入特征的维度
        :param hidden_sizes: 隐藏层每层的神经元数量
        :param dropout: Dropout 概率
        """
        super(AnimeRatingPredictor, self).__init__()

        # 定义全连接层
        self.fc1 = nn.Linear(input_size, hidden_sizes[0])
        self.bn1 = nn.BatchNorm1d(hidden_sizes[0])  # Batch Normalization
        
        self.fc2 = nn.Linear(hidden_sizes[0], hidden_sizes[1])
        self.bn2 = nn.BatchNorm1d(hidden_sizes[1])
        
        self.fc3 = nn.Linear(hidden_sizes[1], hidden_sizes[2])
        self.bn3 = nn.BatchNorm1d(hidden_sizes[2])
        
        # 输出层
        self.output = nn.Linear(hidden_sizes[2], 1)
        
        # Dropout 层
        self.dropout = nn.Dropout(dropout)
    
    def forward(self, x):
        """
        前向传播
        :param x: 输入特征
        :return: 预测评分，范围 [1, 10]
        """
        # 第一层
        x = F.relu(self.bn1(self.fc1(x)))
        x = self.dropout(x)
        
        # 第二层
        x = F.relu(self.bn2(self.fc2(x)))
        x = self.dropout(x)
        
        # 第三层
        x = F.relu(self.bn3(self.fc3(x)))
        x = self.dropout(x)
        
        # 输出层 (限制范围在 [1, 10])
        x = self.output(x)
        x = torch.sigmoid(x) * 9 + 1  # 映射到 [1, 10]
        return x

In [3]:
import os
import sys

# run this block once only
project_root = os.path.abspath("../../")  # 根据文件层级调整路径
os.chdir(project_root)

if project_root not in sys.path:
    sys.path.insert(0, project_root)

import os

# 打印当前工作目录
current_directory = os.getcwd()
print(f"Current working directory: {current_directory}")

Current working directory: c:\Users\admin\Documents\GitHub\MALSugoi


In [4]:
import torch
model_path = "recommenders_DNN/model/model.pth"
model = torch.load(model_path)
model.eval()  # 设置为评估模式
print("Model loaded successfully.")

Model loaded successfully.


C:\Users\admin\AppData\Local\Temp\ipykernel_18848\511128740.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = torch.load(model_path)


In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [6]:
# 2. 加载番剧特征数据
anime_data = pd.read_csv('./data/anime_info/anime_data.csv')

# 清理数据（与之前一致）
anime_data['score'] = anime_data['score'].replace('-', 0).astype(float)
anime_data['members'] = anime_data['members'].str.replace(',', '').astype(float)
anime_data['favorites'] = anime_data['favorites'].str.replace(',', '').astype(float)
anime_data['popularity'] = anime_data['popularity'].str.replace('#', '').astype(float)
anime_data['ranked'] = anime_data['ranked'].str.replace('#', '').replace('-', 0).astype(float)

# 编码 genres
from sklearn.preprocessing import MultiLabelBinarizer, StandardScaler

def preprocess_genres(genres_series):
    genres_series = genres_series.fillna('')
    genres_list = genres_series.str.split(', ')
    mlb = MultiLabelBinarizer()
    genres_encoded = mlb.fit_transform(genres_list)
    return genres_encoded, mlb.classes_

genres_encoded, genres_classes = preprocess_genres(anime_data['genres'])
anime_data = anime_data.join(pd.DataFrame(genres_encoded, columns=genres_classes))

# 特征列
anime_features = ['score', 'ranked', 'popularity', 'members', 'favorites'] + list(genres_classes)

# 标准化特征
scaler = StandardScaler()
anime_data[anime_features] = scaler.fit_transform(anime_data[anime_features])

# 转换为 tensor
anime_tensor = torch.tensor(anime_data[anime_features].values, dtype=torch.float32).to(device)
titles = anime_data['title'].values
title_to_index = {title: idx for idx, title in enumerate(titles)}

# 3. 加载用户评分数据
user_input = pd.read_csv('./recommenders_DNN/test/input.txt', header=None, names=['username', 'anime', 'rating'])
user_input['rating'] = user_input['rating'].replace('-', 0).astype(float)  # 将 '-' 转为 0

# 获取用户的历史评分
user_history = user_input[user_input['rating'] > 0][['anime', 'rating']].set_index('anime')['rating'].to_dict()

In [ ]:
def generate_recommendations(model, user_history, anime_tensor, title_to_index, top_k):
    """
    根据用户历史评分生成推荐
    """
    history_features = []
    for anime, rating in user_history.items():
        if anime in title_to_index:
            anime_idx = title_to_index[anime]
            anime_feature = anime_tensor[anime_idx]
            history_features.append(anime_feature * rating)
    
    # 如果没有用户历史，返回空推荐
    if len(history_features) == 0:
        return []

    # 聚合历史特征（取均值）
    history_features = torch.stack(history_features).mean(dim=0)

    # 对所有未评分的番剧进行预测
    predictions = {}
    for title, idx in title_to_index.items():
        if title not in user_history:  # 只推荐未评分的番剧
            input_features = torch.cat([history_features, anime_tensor[idx]])
            input_features = input_features.unsqueeze(0)  # 增加 batch 维度
            predicted_rating = model(input_features.to(device)).item()
            predictions[title] = predicted_rating

    # 按评分排序，返回前 top_k
    recommended_titles = sorted(predictions.items(), key=lambda x: x[1], reverse=True)[:top_k]
    return recommended_titles

Top 10 推荐的番剧：
1. Shingeki no Kyojin (预测评分: 9.77)
2. Shingeki no Kyojin Season 3 Part 2 (预测评分: 9.70)
3. Shingeki no Kyojin: The Final Season (预测评分: 9.64)
4. Shingeki no Kyojin: The Final Season - Kanketsu-hen (预测评分: 9.62)
5. Shingeki no Kyojin: The Final Season Part 2 (预测评分: 9.62)
6. Code Geass: Hangyaku no Lelouch R2 (预测评分: 9.60)
7. Saiki Kusuo no Ψ-nan (预测评分: 9.60)
8. Saiki Kusuo no Ψ-nan 2 (预测评分: 9.60)
9. Gintama°: Aizome Kaori-hen (预测评分: 9.58)
10. Shingeki no Kyojin Season 3 (预测评分: 9.57)
11. Nichijou (预测评分: 9.57)
12. Koukaku Kidoutai: Stand Alone Complex (预测评分: 9.57)
13. Ping Pong the Animation (预测评分: 9.57)
14. Sakigake!! Cromartie Koukou (预测评分: 9.56)
15. Shingeki no Kyojin Season 2 (预测评分: 9.55)
16. Saiki Kusuo no Ψ-nan: Kanketsu-hen (预测评分: 9.54)
17. Grand Blue (预测评分: 9.53)
18. Tentai Senshi Sunred (预测评分: 9.53)
19. Tentai Senshi Sunred 2nd Season (预测评分: 9.52)
20. Saiki Kusuo no Ψ-nan: Ψ-shidou-hen (预测评分: 9.51)
21. Dungeon Meshi (预测评分: 9.51)
22. Great Teacher Onizuka (预测评分: 9.50)
23.

In [ ]:
# 生成推荐
recommendations = generate_recommendations(model, user_history, anime_tensor, title_to_index, top_k=50)

# 将结果存储到 Pandas DataFrame
recommendations_df = pd.DataFrame(recommendations, columns=["Title", "Predicted Rating"])

print(recommendations_df.head(50))